# Prohledávání stavového prostoru
(Základní techniky umělé inteligence)

In [116]:
# Pomocne funkce a tridy
# TODO: vyfaktorovat do samostatneho modulu
from collections import namedtuple
Position = namedtuple('Position', ['row', 'col'])

class State:
    def __init__(self, world):
        # TODO: enforce immutability
        # TODO: allow for state[position]
        self.world = world
        self.n = max(pos.row for pos in world) + 1
        self.m = max(pos.col for pos in world) + 1
        # finds spaceship
        self.spaceship = None
        for pos, field in world.items():
            if field == 'S':
                self.spaceship = pos
                break
        # We allow for plans without a spaceship
        # (used e.g. by show_path)

    def is_goal(self):
        return self.spaceship.row == 0
        
    def __str__(self):
        # TODO: sjednotit vypis a parsovani (ekvivalentni reprezentace)
        # TODO: also add __repr__ for debugging (State('''...''')
        fields = [[
            self.world[(Position(row, col))]
            for col in range(self.m)]
                for row in range(self.n)]
        text = '\n'.join(''.join(row) for row in fields)
        text = text.replace(' ', '.')
        return text
        
    
    
def perform(state, action):
    # TODO: thow an error if the resulting state is dead (?)
    # (or allow for representation of dead states)
    world = state.world.copy()
    spaceship = move(state.spaceship, action)
    world[state.spaceship] = ' '
    world[spaceship] = 'S' 
    return State(world)

def move(spaceship, action):
    dy = -1
    dx = -1 if action == 'l' else 0 if action == 'f' else 1
    new_spaceship = Position(
        row=spaceship.row + dy,
        col=spaceship.col + dx)
    return new_spaceship    


def actions(state):
    """Return actions that don't lead to dead state.
    """
    return [
        a for a in 'lfr'
        if state.world.get(move(state.spaceship, a), 'A') != 'A']     

        
def parse_state(text):
    rows = text.strip().split('||')
    fields = [row.strip('|').split('|') for row in rows]
    world = {}
    spaceship = None
    for r, row in enumerate(fields):
        for c, field in enumerate(row):
            world[Position(r, c)] = field
            if field == 'S':
                spaceship = Position(r, c)
    return State(world) 
        
        
def show_path(path):
    # path = iterable of States
    # Constructs a pseudo-state showing the path
    world = path[0].world.copy()
    for i, state in enumerate(path):
        world[state.spaceship] = str(i)
    print(State(world))
        


state = parse_state(
'| | | | | |'
'| | | | | |'
'|A|A|A|A| |'
'| | | | | |'
'| | |S| | |')
#print(state)


# stav: pozice raketky (row, col)
# (zbytek sveta je nemenny)

# akce: 'l', 'f', 'r' (left, forward, right)
#print(perform(perform(perform(state, 'r'), 'r'), 'l'))

In [121]:
# TODO: colored-table (or even better: use our react component)
from IPython.display import HTML, display

def show_state(state):
    text = str(state).split('\n')
    html = HTML('''
        <style>
          table {
            border-collapse: collapse;
          }
          table, td {
            border: 1px solid black;
          }   
        </style>
        ''' + 
        '<table><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format(
                    '</td><td>'.join(
                        field for field in row)) for row in text)))
    display(html)
    
show_state(state)

.,.,.,.,.
.,.,.,.,.
A,A,A,A,.
.,.,.,.,.
.,.,S,.,.


.....
.....
AAAA.
.....
..S..


## DFS

In [120]:
# TODO: implement
# def dfs(initial_state):
#     stack = [initial_state]
#     parents = {}
#     while stack:
#         state = stack.pop()
#         if state.is_goal():           
        

In [118]:
# Tree-DFS pomoci rekurze (nehlida zacykleni)
def dfs(state):
    """Return path from state to a goal state"""
    #print('---\nstate:\n' + str(state))
    #input()
    if state.is_goal():
        return [state]
    for action in actions(state):      
        next_state = perform(state, action)
        path = dfs(next_state)
        if path:
            return [state] + path
    return None  # no path found

state = parse_state(
'| | | | | |'
'| | | | | |'
'|A|A|A|A| |'
'| | | | | |'
'| | |S| | |')
show_path(dfs(state))

..4..
...3.
AAAA2
...1.
..0..


## TODO

- intro
- DFS
- DFS - explain tree search vs. graph search
- nastroje pro pohodlne ladeni vsech algoritmu (vizualizace cesty i prubehu planovani (explored/frontier/unexplored states) -> umoznit vypisy (ala logger), vcetne textovych (-> muzu zkopirovat text stavu a vyzkouset si zacit z neho atp.)
- DFS - recursive version
- BFS
- UCS
- A\*
- refaktorovat (zprehlednit, zjednodusit, okomenotvat) kod
- inline react component for visualizing states, paths (and ideally also allow to play the game... or at least provide a link to task-editor)
- rezerva: greedy search, BF, DP, "patnáctka" (sliding tiles)
- parsovani sveta (jako v JS)
- vyfaktorovat pomocné funkce do samostatného .py modulu
- hezčí vykreslování stavu (HTML tabulka, react components)
- another notebook with stripped solutions
- inspirace: Jak to vyresit, Programatorska cvicebnice, Sbirka do Navalu, KSI (napr. Honzovo bludiste, davna DFS/BFS videa), Ucadity AI, EdX AI lecture, google
- testing by friends
- utopicke: pridat jako dalsi level do RoboMise (s pripravenymi high-level bloky a/nebo v RoboKodu)

## Rozšíření
* protihráč -> minimax
* náhoda, nejistota -> expectimax, MDP (value iteration - DP)
* too many states (even infinitely many of them)
* continuous actions
* ...

## Další zdroje
* TBA: videa z AI kurzu na EdX